In [ ]:
pip install sbi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.0/460.0 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
 

In [ ]:
pip install torch


In [ ]:
pip install sbi

In [ ]:
# Example setup
import torch

from sbi.utils import BoxUniform

In [ ]:
# Define the prior
num_dims = 2
num_sims = 1000
num_rounds = 2
prior = BoxUniform(low=torch.zeros(num_dims), high=torch.ones(num_dims))
simulator = lambda theta: theta + torch.randn_like(theta) * 0.1
x_o = torch.tensor([0.5, 0.5])

In [ ]:
from sbi.inference import NPE

In [ ]:
from sbi.inference import NPE

inference = NPE(prior)
theta = prior.sample((num_sims,))
x = simulator(theta)
inference.append_simulations(theta, x).train()
posterior = inference.build_posterior()
samples = posterior.sample((1000,), x=x_o)

 Neural network successfully converged after 131 epochs.

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
from sbi.inference import NPE_A

In [ ]:
from sbi.inference import NPE_A

inference = NPE_A(prior)
proposal = prior
for r in range(num_rounds):
    theta = proposal.sample((num_sims,))
    x = simulator(theta)
    # NPE-A trains a Gaussian density estimator in all but the last round. In the last round,
    # it trains a mixture of Gaussians, which is why we have to pass the `final_round` flag.
    final_round = r == num_rounds - 1
    _ = inference.append_simulations(theta, x, proposal=proposal).train(final_round=final_round)
    posterior = inference.build_posterior().set_default_x(x_o)
    proposal = posterior

 Neural network successfully converged after 70 epochs.

  0%|          | 0/1000 [00:00<?, ?it/s]

 Neural network successfully converged after 38 epochs.

In [ ]:
from sbi.inference import NPE_B
from sbi.neural_nets import posterior_nn
from sbi.neural_nets.embedding_nets import FCEmbedding

In [ ]:
embedding = FCEmbedding(num_dims)
density_estimator = posterior_nn("mdn", embedding_net=embedding)
inference = NPE_B(prior, density_estimator=density_estimator)

proposal = prior
for _ in range(num_rounds):
    theta = proposal.sample((num_sims,))
    x = simulator(theta)
    _ = inference.append_simulations(theta, x, proposal=proposal).train()
    posterior = inference.build_posterior().set_default_x(x_o)
    proposal = posterior

 Neural network successfully converged after 90 epochs.

  0%|          | 0/1000 [00:00<?, ?it/s]

 Neural network successfully converged after 23 epochs.

In [ ]:
from sbi.inference import NPE
from sbi.neural_nets import posterior_nn
from sbi.neural_nets.embedding_nets import FCEmbedding


In [ ]:
from sbi.inference import NPE
from sbi.neural_nets import posterior_nn
from sbi.neural_nets.embedding_nets import FCEmbedding

embedding = FCEmbedding(num_dims)
density_estimator = posterior_nn("maf", embedding_net=embedding)
inference = NPE(prior, density_estimator=density_estimator)

proposal = prior
for _ in range(num_rounds):
    theta = proposal.sample((num_sims,))
    x = simulator(theta)
    _ = inference.append_simulations(theta, x, proposal=proposal).train()
    posterior = inference.build_posterior().set_default_x(x_o)
    proposal = posterior


 Neural network successfully converged after 376 epochs.

  0%|          | 0/1000 [00:00<?, ?it/s]

Using SNPE-C with atomic loss
 Neural network successfully converged after 164 epochs.

In [ ]:
# Posterior estimation with BayesFlow is equivalent to single-round NPE.
from sbi.inference import NPE
from sbi.neural_nets import posterior_nn
from sbi.neural_nets.embedding_nets import FCEmbedding, PermutationInvariantEmbedding

In [ ]:
# Posterior estimation with BayesFlow is equivalent to single-round NPE.
from sbi.inference import NPE
from sbi.neural_nets import posterior_nn
from sbi.neural_nets.embedding_nets import FCEmbedding, PermutationInvariantEmbedding

num_iid = 3
simulator_iid = lambda theta: theta + torch.randn((num_iid, *theta.shape)) * 0.1

trial_net = FCEmbedding(num_dims, 20)
embedding = PermutationInvariantEmbedding(trial_net, 20)
density_estimator = posterior_nn("maf", embedding_net=embedding)
inference = NPE(prior, density_estimator=density_estimator)

theta = prior.sample((num_sims,))
x = simulator_iid(theta).permute(1, 0, 2)
inference.append_simulations(theta, x).train()
posterior = inference.build_posterior()
samples = posterior.sample((1000,), x=torch.zeros((1, num_iid, num_dims)))

 Neural network successfully converged after 134 epochs.

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
from sbi.inference import NPE
from sbi.utils import RestrictedPrior, get_density_thresholder

In [ ]:
from sbi.inference import NPE
from sbi.utils import RestrictedPrior, get_density_thresholder

inference = NPE(prior)
proposal = prior
for _ in range(num_rounds):
    theta = proposal.sample((num_sims,))
    x = simulator(theta)
    _ = inference.append_simulations(theta, x).train(force_first_round_loss=True)
    posterior = inference.build_posterior().set_default_x(x_o)

    accept_reject_fn = get_density_thresholder(posterior, quantile=1e-4)
    proposal = RestrictedPrior(prior, accept_reject_fn, sample_with="rejection")

 Neural network successfully converged after 207 epochs.

  0%|          | 0/1000000 [00:00<?, ?it/s]

The `RestrictedPrior` rejected 49.7% of prior samples. You will get a speed-up of 98.7%.
 Neural network successfully converged after 29 epochs.

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [ ]:
from sbi.inference import FMPE

inference = FMPE(prior)
# FMPE does not support multiple rounds of inference
theta = prior.sample((num_sims,))
x = simulator(theta)
inference.append_simulations(theta, x).train()
posterior = inference.build_posterior().set_default_x(x_o)

 Neural network successfully converged after 169 epochs.

In [ ]:
from sbi.inference import NPSE

theta = prior.sample((num_sims,))
x = simulator(theta)

inference = NPSE(prior, sde_type="ve")
_ = inference.append_simulations(theta, x).train()
posterior = inference.build_posterior().set_default_x(x_o)

 Neural network successfully converged after 140 epochs.